# Importing required modules 

In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder

### Loading data

In [2]:
train_data = pd.read_csv("../input/30-days-of-ml/train.csv")
test_data = pd.read_csv("../input/30-days-of-ml/test.csv")
sample_submission = pd.read_csv("../input/30-days-of-ml/sample_submission.csv")

### K-Fold model
* having 5 splits

In [3]:
train_data["kfold"] = -1
Kf_model = KFold(n_splits=5, random_state=1, shuffle=True)
# fold (0, 1, 2, 3, 4); train_index (0, 2, 3...); valid_index(1, 4, 6)
for fold, (train_index, valid_index) in enumerate(Kf_model.split(X=train_data)):
    train_data.loc[valid_index, "kfold"] = fold

#### Training data after making K-fold column using K-fold model

In [4]:
print(train_data.shape)
train_data.head()

(300000, 27)


,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target,kfold
0,1,B,B,B,C,B,B,A,E,C,...,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634,4
1,2,B,B,A,A,B,D,A,F,A,...,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233,0
2,3,A,A,A,C,B,D,A,D,A,...,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351,4
3,4,B,B,A,C,B,D,A,E,C,...,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253,1
4,6,A,A,A,C,B,D,A,E,A,...,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260,0


#### Setting up columns
1. num_cols stores all columns of training data which have numerical columns
2. cat_cols stores all columns of training data which have categorical columns
3. useful_cols stores both num_cols and cat_cols
* useful_cols are required for target encoding, hence test_data also uses the useful_cols only

In [5]:
num_cols = [col for col in train_data.columns if "cont" in col]
cat_cols = [col for col in train_data.columns if "cat" in col]
useful_cols = cat_cols + num_cols
test_data = test_data[useful_cols]

### Target Encoding

In [6]:
# Iterate over categorical columns
for col in cat_cols:
    """
    Based on each categorical column, one target fold is created
    total 5 folds for one column
    """
    temp_train = []
    temp_test_target = None
    for fold in range(5):
        # making training data and validating data for each fold
        X_train = train_data[train_data.kfold != fold].reset_index(drop=True)
        X_valid = train_data[train_data.kfold == fold].reset_index(drop=True)
        
        # getting the mean of training data target
        mean_target = dict(X_train.groupby(col)["target"].agg("mean"))
        
        # adding the mean_target to X_valid (valid dataset of each fold)
        X_valid.loc[:, f"target_enc_{col}"] = X_valid[col].map(mean_target)
        temp_train.append(X_valid)
        if (temp_test_target is None):
            temp_test_target = test_data[col].map(mean_target)
        else:
            temp_test_target += test_data[col].map(mean_target)
        
    # getting the average of temporary test target on each column
    temp_test_target /= 5
    # adding the temporary test target to test data on each column (total=10)
    test_data.loc[:, f"target_enc_{col}"] = temp_test_target
    
    # setting training data as temp_train
    train_data = pd.concat(temp_train)

#### Training data after Target Encoding

In [7]:
train_data.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,target_enc_cat0,target_enc_cat1,target_enc_cat2,target_enc_cat3,target_enc_cat4,target_enc_cat5,target_enc_cat6,target_enc_cat7,target_enc_cat8,target_enc_cat9
0,2,B,B,A,A,B,D,A,F,A,...,8.246824,8.204869,8.246016,8.277317,8.241689,8.251699,8.241846,8.300596,8.233186,8.241516
1,6,A,A,A,C,B,D,A,E,A,...,8.240298,8.278027,8.246016,8.237701,8.241689,8.251699,8.241846,8.241403,8.233186,8.253157
2,8,B,A,A,A,B,D,A,E,C,...,8.246824,8.278027,8.246016,8.277317,8.241689,8.251699,8.241846,8.241403,8.281416,8.260024
3,10,A,B,A,C,B,D,A,E,G,...,8.240298,8.204869,8.246016,8.237701,8.241689,8.251699,8.241846,8.241403,8.252384,8.224612
4,18,B,A,A,C,B,D,A,E,A,...,8.246824,8.278027,8.246016,8.237701,8.241689,8.251699,8.241846,8.241403,8.233186,8.233269


#### Test data after Target Encoding

In [8]:
test_data.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,target_enc_cat0,target_enc_cat1,target_enc_cat2,target_enc_cat3,target_enc_cat4,target_enc_cat5,target_enc_cat6,target_enc_cat7,target_enc_cat8,target_enc_cat9
0,B,B,B,C,B,B,A,E,E,I,...,8.246840,8.204026,8.224860,8.237026,8.240946,8.230315,8.241024,8.24063,8.193336,8.223862
1,A,B,A,C,B,C,A,E,C,H,...,8.239288,8.204026,8.245088,8.237026,8.240946,8.268637,8.241024,8.24063,8.282179,8.240703
2,B,A,A,A,B,B,A,E,D,K,...,8.246840,8.277565,8.245088,8.276706,8.240946,8.230315,8.241024,8.24063,8.242740,8.269897
3,B,B,A,C,B,D,A,E,A,N,...,8.246840,8.204026,8.245088,8.237026,8.240946,8.251199,8.241024,8.24063,8.230476,8.249954
4,B,B,A,C,B,C,A,E,C,F,...,8.246840,8.204026,8.245088,8.237026,8.240946,8.268637,8.241024,8.24063,8.282179,8.259046


#### Remaking the columns for final predictions

In [9]:
num_cols = [col for col in train_data.columns if "cont" in col]
cat_cols = [col for col in train_data.columns if col.startswith("cat")]
useful_cols = cat_cols + num_cols
test_data = test_data[useful_cols]

### Getting final predictions using XGB Regressor model on each fold
* **final_predictions** and **MSE** on each fold

In [10]:
final_predictions = []
mse_scores = []

# As total folds are 5, getting predictions and rmse at every fold
for fold in range(5):
    # Splitting the training and validating data according to fold and kfold
    X_train = train_data[train_data.kfold != fold].reset_index(drop=True)
    X_valid = train_data[train_data.kfold == fold].reset_index(drop=True)
    
    X_test = test_data.copy()
    
    # setting the target
    y_train = X_train.target
    y_valid = X_valid.target
    
    # setting the training data and validating data
    X_train = X_train[useful_cols]
    X_valid = X_valid[useful_cols]
    
    # One Hot Encoding on categorical data
    OH_encoder = OneHotEncoder(handle_unknown="ignore", sparse=False)
    
    # fitting and transforming the training and test data
    OH_encoder.fit(X_train[cat_cols])
    OH_X_train = OH_encoder.transform(X_train[cat_cols])
    OH_X_valid = OH_encoder.transform(X_valid[cat_cols])
    OH_X_test = OH_encoder.transform(X_test[cat_cols])
    
    # Naming the one hot encoded columns
    OH_X_train = pd.DataFrame(OH_X_train, columns=[f"ohe_{i}" for i in range(OH_X_train.shape[1])])
    OH_X_valid = pd.DataFrame(OH_X_valid, columns=[f"ohe_{i}" for i in range(OH_X_valid.shape[1])])
    OH_X_test = pd.DataFrame(OH_X_test, columns=[f"ohe_{i}" for i in range(OH_X_test.shape[1])])
    
    # Adding one hot encoded columns to main data (training, validating, test)
    X_train = pd.concat([X_train, OH_X_train], axis=1)
    X_valid = pd.concat([X_valid, OH_X_valid], axis=1)
    X_test = pd.concat([X_test, OH_X_test], axis=1)
    
    # Dropping the categorical columns, as their one hot encoded columns are added to main data
    X_train = X_train.drop(cat_cols, axis=1)
    X_valid = X_valid.drop(cat_cols, axis=1)
    X_test = X_test.drop(cat_cols, axis=1)
    
    # Making XGBoost Regressor model
    model = XGBRegressor(n_estimators=1000, learning_rate=0.05, random_state=fold*10, tree_method='gpu_hist', gpu_id=0, predictor="gpu_predictor")
    model.fit(X_train, y_train, early_stopping_rounds=5, eval_set=[(X_valid, y_valid)], verbose=False)
    
    # pridicting the target on training and test data
    train_preds = model.predict(X_valid)
    test_preds = model.predict(X_test)
    final_predictions.append(test_preds)
    
    # MSE at each prediction
    mse = mean_squared_error(y_valid, train_preds, squared=False)
    mse_scores.append(mse)
    
    print(f"Fold: {fold}, RMSE: {mse}, \nPredictions on test data: {test_preds}")

Fold: 0, RMSE: 0.7199421488116169, 
Predictions on test data: [7.97914  8.318346 8.361334 ... 8.405242 8.159172 8.015999]
Fold: 1, RMSE: 0.7273550071119027, 
Predictions on test data: [8.035237 8.317371 8.344818 ... 8.355633 8.034177 8.005104]
Fold: 2, RMSE: 0.7227010220830685, 
Predictions on test data: [8.076327 8.395693 8.354329 ... 8.379021 8.047301 8.038431]
Fold: 3, RMSE: 0.7217158903027857, 
Predictions on test data: [8.003313  8.340549  8.379889  ... 8.353674  8.108551  7.8833833]
Fold: 4, RMSE: 0.7173706486251307, 
Predictions on test data: [8.044944  8.385874  8.393736  ... 8.3408165 8.172313  8.070282 ]


In [11]:
final_preds = np.mean(np.column_stack(final_predictions), axis=1)
final_preds

array([8.027792, 8.351567, 8.366821, ..., 8.366877, 8.104303, 8.00264 ],
      dtype=float32)

### Final predictions on test data

In [12]:
# Submitting predicted target to `submission.csv`
output = pd.DataFrame({'Id': sample_submission.id,'target': final_preds})
output.to_csv('submission.csv', index=False)